In [5]:
# import dependicies for this 
import pandas as pd
import numpy as np 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor   
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from xgboost import XGBRegressor


In [8]:
df=pd.read_csv(r'data\stud.csv')

In [9]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [10]:
X=df.drop('math_score',axis=1)
Y=df['math_score']

In [11]:
num_features=X.select_dtypes(exclude='object').columns
cat_features=X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transfomer=OneHotEncoder()

preprocessor=ColumnTransformer([('OneHotEncoder',oh_transfomer,cat_features),
                                ('StandardScaler',numeric_transformer,num_features),
                                ])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   math_score                   1000 non-null   int64 
 6   reading_score                1000 non-null   int64 
 7   writing_score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [13]:
X=preprocessor.fit_transform(X)

In [14]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [15]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=45)

In [16]:
len(x_train)

800

In [17]:
def model_evaluate(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    r2=r2_score(true,predicted)
    return mae,mse,r2




In [18]:
models={
    'linearregrssion':LinearRegression(),
    'decisiontree':DecisionTreeRegressor(),
    'randomforest':RandomForestRegressor(),
    'xgboost':CatBoostRegressor(),
    'catboost':XGBRegressor()
}

In [19]:
model_list=[]
r2_score_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)
    # make predicton
    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)
    # evaluate train and test data set
    model_train_mae, model_train_mse, model_train_r2 = model_evaluate(y_train, y_train_pred)
    model_test_mae,model_test_mse,model_test_r2=model_evaluate(y_test,y_test_pred)

    print(list(models.values())[i])
    model_list.append(list(models.values())[i])
    print('model performance for training')
    print("mean absolute error :{:.4f}".format(model_train_mae))
    print("mean squared error :{:.4f}".format(model_train_mse))
    print("r2 score :{:.4f}".format(model_train_r2))
    print('-'*50)
    print('model performance for test')
    print("mean absolute error :{:.4f}".format(model_test_mae))
    print("mean squared error :{:.4f}".format(model_test_mse))
    print("r2 score :{:.4f}".format(model_test_r2))
    r2_score_list.append(model_test_r2)
    print("="*50)
    print('\n')

LinearRegression()
model performance for training
mean absolute error :4.2156
mean squared error :28.2187
r2 score :0.8792
--------------------------------------------------
model performance for test
mean absolute error :4.3716
mean squared error :29.3141
r2 score :0.8610


DecisionTreeRegressor()
model performance for training
mean absolute error :0.0262
mean squared error :0.1406
r2 score :0.9994
--------------------------------------------------
model performance for test
mean absolute error :7.2350
mean squared error :77.7450
r2 score :0.6314


RandomForestRegressor()
model performance for training
mean absolute error :1.8184
mean squared error :5.3042
r2 score :0.9773
--------------------------------------------------
model performance for test
mean absolute error :5.0308
mean squared error :38.3792
r2 score :0.8181


Learning rate set to 0.039525
0:	learn: 14.8720036	total: 158ms	remaining: 2m 37s
1:	learn: 14.4819737	total: 158ms	remaining: 1m 19s
2:	learn: 14.0977389	total: 15

In [20]:
pd.DataFrame(list(zip(models,r2_score_list)),columns=['models','r2_score_list'])

,models,r2_score_list
0,linearregrssion,0.861030
1,decisiontree,0.631433
2,randomforest,0.818055
3,xgboost,0.827945
4,catboost,0.796809


In [21]:
pd.DataFrame({'actualvalues':y_test,'predcited':y_test_pred,'difference':y_test-y_test_pred})

,actualvalues,predcited,difference
726,65,66.497200,-1.497200
243,59,49.986183,9.013817
342,69,63.181313,5.818687
976,60,64.964432,-4.964432
919,91,94.540184,-3.540184
...,...,...,...
458,100,97.292397,2.707603
169,67,56.928753,10.071247
297,71,79.957947,-8.957947
10,58,57.383270,0.616730
